# 1_prepare_data

A notebook to prepare the data for the tests

In [1]:
from datetime import datetime

import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
dfp_consumption = pd.read_csv('./data/rtu/rte_daily_consumption.csv')
dfp_weather = pd.read_csv('./data/rtu/nasa_weather.csv')

In [3]:
dfp_consumption.head()

,date,daily_electrical_consumption
0,2015-01-01,3227643.0
1,2015-01-02,3273903.0
2,2015-01-03,2787401.0
3,2015-01-04,2912191.0
4,2015-01-05,2152950.0


In [4]:
dfp_weather.head()

,date,t2m_min_bordeaux,t2m_bordeaux,t2m_max_bordeaux,prectot_bordeaux,t2m_min_lille,t2m_lille,t2m_max_lille,prectot_lille,t2m_min_paris,...,t2m_max_nice,prectot_nice,t2m_min_strasbourg,t2m_strasbourg,t2m_max_strasbourg,prectot_strasbourg,t2m_min_montpellier,t2m_montpellier,t2m_max_montpellier,prectot_montpellier
0,2015-01-01,-2.34,1.25,6.17,0.01,-1.38,0.79,3.84,0.08,-2.69,...,7.70,0.00,-2.39,-1.08,1.86,0.08,-1.41,1.78,7.06,0.01
1,2015-01-02,-0.46,3.54,7.35,0.31,0.52,4.89,8.45,2.39,-1.09,...,10.26,0.00,-3.94,-0.81,1.22,3.64,1.32,4.99,10.52,0.00
2,2015-01-03,5.77,9.41,12.72,1.17,0.32,2.12,5.32,12.33,-1.02,...,11.16,0.00,-1.78,1.13,5.78,13.79,4.88,7.71,12.36,0.22
3,2015-01-04,5.78,10.87,13.47,1.22,-1.81,1.45,5.22,0.05,-2.06,...,12.71,0.00,-2.48,-1.18,2.30,0.40,3.07,8.02,12.51,0.60
4,2015-01-05,3.02,5.25,9.27,0.05,-0.61,0.79,2.93,0.20,-2.80,...,10.90,0.01,-3.56,-1.18,2.97,0.03,1.54,4.24,10.05,0.00


In [26]:
# Merge the electrical consumption and the weather data
dfp_data = pd.merge(dfp_consumption, dfp_weather, on =['date'])
dfp_data['date'] = pd.to_datetime(dfp_data['date'])

In [ ]:
# Compute averaged temperature and precipatation bassed on the population of the top11 cities in France
dict_cities_population = {
    'bordeaux' : 257068,
    'lille' : 233098,
    'paris' : 2175601,
    'rennes' : 217728,
    'nantes' : 314138,
    'toulouse' : 486828,
    'marseille' : 868277,
    'lyon' : 518635,
    'nice' : 341138,
    'strasbourg' : 284677,
    'montpellier' : 290053
}

sum_population_cities = sum([value for key, value in dict_cities_population.items()])
for parameter in ['t2m', 't2m_min', 't2m_max', 'prectot']:
    dfp_data[f'weighted_{parameter}'] = dfp_data.apply(lambda row: sum([1.0 * (value/sum_population_cities) * row[f'{parameter}_{key}'] for key, value in dict_cities_population.items()]), axis=1)

In [27]:
# Add feature related to the day
dfp_data['weekday'] = dfp_data['date'].apply(lambda date: date.weekday())
dfp_data['month'] = dfp_data['date'].apply(lambda date: date.month)
dfp_data['week_number'] = dfp_data['date'].apply(lambda date: date.week)

In [28]:
dfp_data.head()

,date,daily_electrical_consumption,t2m_min_bordeaux,t2m_bordeaux,t2m_max_bordeaux,prectot_bordeaux,t2m_min_lille,t2m_lille,t2m_max_lille,prectot_lille,...,t2m_montpellier,t2m_max_montpellier,prectot_montpellier,weighted_t2m,weighted_t2m_min,weighted_t2m_max,weighted_prectot,weekday,month,week_number
0,2015-01-01,3227643.0,-2.34,1.25,6.17,0.01,-1.38,0.79,3.84,0.08,...,1.78,7.06,0.01,1.358532,-1.254460,5.330187,0.017693,3,1,1
1,2015-01-02,3273903.0,-0.46,3.54,7.35,0.31,0.52,4.89,8.45,2.39,...,4.99,10.52,0.00,3.752924,0.624573,7.817334,0.815847,4,1,1
2,2015-01-03,2787401.0,5.77,9.41,12.72,1.17,0.32,2.12,5.32,12.33,...,7.71,12.36,0.22,6.726723,2.088169,10.977501,3.666066,5,1,1
3,2015-01-04,2912191.0,5.78,10.87,13.47,1.22,-1.81,1.45,5.22,0.05,...,8.02,12.51,0.60,5.330859,1.823681,9.144834,0.573484,6,1,1
4,2015-01-05,2152950.0,3.02,5.25,9.27,0.05,-0.61,0.79,2.93,0.20,...,4.24,10.05,0.00,2.890455,0.386511,6.789996,0.072451,0,1,2


In [29]:
# Split all the data that was before the 1st of January 2020 and the one after
dfp_data_model = dfp_data[(dfp_data['date'] < datetime(2020, 1, 1))]
dfp_data_2020 = dfp_data[(dfp_data['date'] >= datetime(2020, 1, 1))]

In [30]:
# Sae the data in files
dfp_data_model.to_csv('./data/rtu/model_data.csv', index=None)
dfp_data_2020.to_csv('./data/rtu/2020_data.csv', index=None)

In [31]:
# Build a training and testing set (save them in files)
dfp_train, dfp_test = train_test_split(dfp_data_model, test_size=0.2, random_state=0)
dfp_train.to_csv('./data/rtu/model_train_data.csv', index=None)
dfp_test.to_csv('./data/rtu/model_test_data.csv', index=None)